In [2]:
!pip install transformers datasets accelerate


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.9 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset


In [4]:
dataset = load_dataset("imdb")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [5]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)


In [9]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.224700,0.212688
2,0.151100,0.229000


Epoch,Training Loss,Validation Loss
1,0.224700,0.212688
2,0.151100,0.229000
3,0.094000,0.273131


TrainOutput(global_step=4689, training_loss=0.1676473564673497, metrics={'train_runtime': 4897.8782, 'train_samples_per_second': 15.313, 'train_steps_per_second': 0.957, 'total_flos': 9935054899200000.0, 'train_loss': 0.1676473564673497, 'epoch': 3.0})

In [10]:
# Evaluate the model
eval_result = trainer.evaluate()
print(f"Evaluation results: {eval_result}")


Evaluation results: {'eval_loss': 0.27313143014907837, 'eval_runtime': 418.916, 'eval_samples_per_second': 59.678, 'eval_steps_per_second': 0.933, 'epoch': 3.0}


In [12]:
import torch

# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Move the model to the appropriate device
model.to(device)

# Example input for predictions
texts = ["I loved the movie!", "It was a terrible film."]

# Tokenize the input texts and move them to the GPU
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)

# Get predictions
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Print the predictions
for text, pred in zip(texts, predictions):
    print(f"Text: {text} | Prediction: {pred.item()}")


Text: I loved the movie! | Prediction: 1
Text: It was a terrible film. | Prediction: 0


In [13]:
model.save_pretrained("path/to/save/model")
tokenizer.save_pretrained("path/to/save/model")


('path/to/save/model/tokenizer_config.json',
 'path/to/save/model/special_tokens_map.json',
 'path/to/save/model/vocab.txt',
 'path/to/save/model/added_tokens.json',
 'path/to/save/model/tokenizer.json')

In [14]:
import os

# Define the directory where you want to save your model
save_directory = r'C:\Users\aaris\fine tuning'  # Use raw string or double backslashes

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Save the model
model.save_pretrained(save_directory)

# Save the tokenizer
tokenizer.save_pretrained(save_directory)

print(f'Model and tokenizer saved in {save_directory}')


Model and tokenizer saved in C:\Users\aaris\fine tuning


In [15]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the fine-tuned model
model = DistilBertForSequenceClassification.from_pretrained(save_directory)

# Load the tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(save_directory)

print('Model and tokenizer loaded successfully.')


Model and tokenizer loaded successfully.
